
[Stochastic Neighbour embedding](https://papers.nips.cc/paper/2276-stochastic-neighbor-embedding)


Briefly, SNE is described below

1) Goal  - To convert m points/samples in high dimensional space x<sup>1</sup>, x<sup>2</sup>.. x<sup>m</sup>  to a lower dimensional space y<sup>1</sup>, y<sup>2</sup>.. y<sup>m</sup> such that they can be better visualized

2) Uses pairwise similarity - A pair of points x<sup>i</sup> and x<sup>j</sup> are mapped to a pair of points y<sup>i</sup> and y<sup>j</sup> such that the <i><b>similarity</b></i> between the points in the high and low dimension is preserved

3) How is <i><b>similarity</b></i> defined ?
The similarity of two points in high dimensional space x<sup>i</sup> and x<sup>j</sup> is the <b>conditional probability</b> p<sub>j|i</sub> that x<sup>i</sup> would pick x<sup>j</sup> as a neighbour; if neighbours were picked under a gaussian distribution centered on x<sup>i</sup>
Mathematically,

p<sub>j|i</sub> = $\frac{e^{-\frac{|x^{i}-x^{j}|^{2}}{2\sigma_i^{2}}}}{\sum_{k,k!=i}{e^{-\frac{|x^{i}-x^{k}|^{2}}{2\sigma_i^{2}}}}}$

4) Similarly , the similarity of points in low dimension space is defined as

q<sub>j|i</sub> = $\frac{e^{-|y^{i}-y^{j}|^{2}}}{\sum_{k,k!=i}{e^{-{|y^{i}-y^{k}|^{2}}}}}$

  (Note that in low dimensional space, $\sigma_i$ does not exist)

5) We define cost function which we want to minimize as the sum of  KL divergence between $P_i$ and $Q_i$

    C = $\sum_iKL(P_i/Q_i)$
      = $\sum_i\sum_jp_{j|i}*ln\frac{p_{j|i}}{q_{i|i}}$


  Therefore, goal of optimization to arrive at y<sup>i</sup>, i=1..m in such a way that this cost function is minimized


6) Note that the above cost is not symmetric. In particular, there is a large cost if $q_{j|i}$ is small and $p_{j|i}$ is large than the other way round -> if points are close together in original space and far apart in low dimensional space, they are penalized; wherease only small cost if points are far in original space but close in lower dimensional space, not penalized so much

6) How is $\sigma_i$ chosen ? Given perplexity (which is equivalent to no of effective neighbours for every point) as an input to the SNE algorithm, we can choose $\sigma_i$ for every point x<sup>i</sup> in such a way that same perplexity is maintained for all sites (choose less no of points if x<sup>i</sup> has more neighbours and more if x<sup>i</sup> has less)

This is implemented algorithmically by using a binary search to find right value of $\sigma_i$ for every i. $Perplexity_i = 2^{H_i}$ where H is the Shannon entropy ; $H_i = -\sum_{j}P_{j|i}ln(P_{j|i})$

7) Minimize C by gradient descent.   $y^{i}_{t+1} = y^{i}_{t} - learningrate*\frac{\partial C}{\partial y^i}$ for all i . (Of course, nowadays, you can choose computational graph formats like pytorch :) )

If you choose to use gradient descent, the gradient has a very simple form:
$\frac{\partial C}{\partial y_i} = 2*\sum_j(p_{j|i}-q_{j|i} + p_{i|j} - q_{i|j})(y_i - y_j)$

(Derivation of this - TBD)
(indices have been changed from i,j to k,l for easier derivative computation - handling indices while taking derivative wrt $y_i$)

C = $\sum_k\sum_lp_{l|k}*ln\frac{p_{l|k}}{q_{l|k}}$
  = $\sum_k\sum_lp_{l|k}*lnp_{l|k} - \sum_k\sum_lp_{l|k}*ln{q_{l|k}}$

Let C1 = $\sum_k\sum_lp_{l|k}*lnp_{l|k}$.
This is a term independent of $y_i$ (no $q$ term), does not matter for derivative

From definition, q<sub>l|k</sub> = $\frac{e^{-|y^{k}-y^{l}|^{2}}}{\sum_{t,t!=k}{e^{-{|y^{k}-y^{t}|^{2}}}}}$

Let q<sub>l|k</sub> = $\frac{D_{kl}}{Z_k}$ where
$D_{kl} = e^{-|y^{k}-y^{l}|^{2}}$, $Z_k = \sum_{t,t!=k}{e^{-{|y^{k}-y^{t}|^{2}}}}$

$\sum_k\sum_lp_{l|k}*ln{q_{l|k}}$ =

$\sum_k\sum_kp_{l|k}*ln{D_{kl}}$ - $\sum_k\sum_lp_{l|k}*ln{Z_k}$

Therefore, C = C1 - $\sum_k\sum_lp_{l|k}*ln{D_{kl}}$ + $\sum_k\sum_lp_{l|k}*ln{Z_k}$


C = C1 - $\sum_k\sum_lp_{l|k}*ln{D_{kl}}$ +
$\sum_kln{Z_k}\sum_lp_{l|k}$   (as $Z_k$ is independent of l)

Note that $\sum_lp_{l|k}$ = 1 for all k


Therefore, C = C1 - $\sum_k\sum_lp_{l|k}*ln{D_{kl}}$   + $\sum_kln{Z_k}$


**Partial Derivative of Term 1 wrt $y_i$ - $\frac {\partial C_1}{y_i}$**

$\frac {\partial C_1}{y_i}$ = 0 as C1 is independent of i


**Partial derivative of term 2 wrt y_i**

In, $\sum_k\sum_lp_{l|k}*ln{D_{kl}}$ the only terms which depend on i are
$\sum_kp_{i|k}lnD_{ik}  + \sum_lp_{l|i}lnD_{il}$, which replacing k and l by j is equivalent to

$\sum_j (p_{i|j}lnD_{ji}  + p_{j|i}lnD_{ij})$

Note that $\frac{\partial D_{ij}}{\partial y_i}$ = $D_{ij}*-2*(y_i - y_j)$

Therefore,
$\frac{\partial term2}{\partial y_i}$ = $\frac{\partial \sum_j (p_{i|j}lnD_{ji}  + p_{j|i}lnD_{ij})}{\partial y_i}$

= $\sum_j(p_{i|j}*-2*(y_i - y_j) + p_{j|i}*-2*(y_j - y_i))$


**Partial derivative of term 3 wrt y_i** - changing k to j for index notation dor similarity with step 2 derivative
$\frac{\partial \sum_jln{Z_j}}{\partial y_i}$

= $\sum_k\frac{1}{Z_k}*\frac{\partial Z_k}{\partial y_i}$

Note that $\frac{\partial Z_k}{\partial y_i}$ =



$\frac{\partial \sum_{t,t!=k}{e^{-{|y^{k}-y^{t}|^{2}}}}}{\partial y_i}$

The only terms here which depend on $y_i$ are 1) when k = i  i.e.  ${\frac{1}{Z_i}\frac{\partial Z_i}{\partial y_i}}$}$ for all t and 2) when k is iterated over all values except i,

$\sum_{k!=i}e^{-|y_k-y_i|^2}$ for all k


Therefore,  $\frac{\partial Z_k}{\partial y_i}$   reduces to (replacing t and k by j for convenience in 1) and 2) )



$\sum_k\frac{1}{Z_k}*\frac{\partial Z_k}{\partial y_i}$

= $\sum_i\frac{1}{Z_k}*\frac{\partial Z_k}{\partial y_i}$








$\frac {\partial \sum_kln{Z_k} }{\partial y_i}$ = $\frac {\partial ln{Z_i} }{\partial y_i}$  (as all other terms are independent of y_i)

= $\frac{1}{Z_i}\frac{\partial Z_i}{\partial y_i}$

$\frac{\partial Z_i}{\partial y_i}$ = $\frac{\partial \sum_{t,t!=i}{e^{-{|y^{i}-y^{t}|^{2}}}}}{\partial y_i}$

= $\sum_{t,t!=i}\frac{\partial e^{-|y^{i}-y^{t}|^{2}}}{\partial y_i}$


$\frac{\partial e^{-|y^{i}-y^{t}|^{2}}}{\partial y_i}$ = $e^{-|y^{i}-y^{t}|^{2}}$*${-2}*(y^{i}-y^{t})$

=>

$\frac{\partial Z_i}{\partial y_i}$ = $\sum_{t,t!=i}e^{-|y^{i}-y^{t}|^{2}}$*${-2}*(y^{i}-y^{t})$

$\frac{\partial C}{\partial y_i}$ =













8) Practical details of implementation
  i)  A large momentum term is added to increase convergence rate
  ii) In early stages of optimization, gaussian noise is added to the map points after each iteration. This loss is slowly reduced with time -  a la simulated annealing - Algorithm sensitive to initial amount of gaussian noise and rate at which it decays. Run algorithm several times with different values of hyper-parameters (momentum parameters, learning rate,  and amount of momentum) - Therefore, no convex optimization, inferior to methods which allow convex optimization

